### KNN Classifier

In [6]:
import numpy as np
import pandas as pd
from scipy import sparse
import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import re, string, time
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import log_loss

Custom functions for loading and preprocessing data.

In [2]:
def most_freq_vects(docs, max_feature=None, percent=None, token_pattern=u'(?u)\b\w\w+\b'):
    vect = CountVectorizer(token_pattern=token_pattern)
    feat_sparse = vect.fit_transform(docs.values.astype('U'))
    freq_table = list(zip(vect.get_feature_names(), np.asarray(feat_sparse.sum(axis=0)).ravel()))
    freq_table = pd.DataFrame(freq_table, columns=['feature', 'count']).sort_values('count', ascending=False)
    if not max_feature:
        if percent:
            max_feature = int(percent * len(vect.get_feature_names()))
        else:
            max_feature = len(vect.get_feature_names())
    feat_df = pd.DataFrame(feat_sparse.todense(), columns=vect.get_feature_names())
    names = list(freq_table.feature[:max_feature])
    return feat_df[names]


def load_data():
    print('Loading features files')
    basic_feat = pd.read_json('../feat_input/basic_feat.json')
    longtime_feat = pd.read_csv('../feat_input/longtime_feat.csv')
    encoded_feat = pd.read_csv('../feat_input/feat_stats_encoding.csv')

    # apply ordinal encoding to categorical feature
    print('Ordinal encoding')
    basic_feat.display_address = basic_feat.display_address.replace(r'\r$', '', regex=True)
    basic_feat.street_address = basic_feat.street_address.replace(r'\r$', '', regex=True)
    categorical = ["display_address", "manager_id", "building_id", "street_address"]
    for f in categorical:
        if basic_feat[f].dtype == 'object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(basic_feat[f].values))
            basic_feat[f] = lbl.transform(list(basic_feat[f].values))

    all_feat = basic_feat.merge(longtime_feat, on='listing_id')
    all_feat = all_feat.merge(encoded_feat, on='listing_id')

    print("Features document-term matrix")
    stemmer = SnowballStemmer('english')
    punct = string.punctuation
    punct = re.sub("'|-", "", punct)
    pattern = r"[0-9]|[{}]".format(punct)
    all_feat['features'] = all_feat['features'].apply(lambda x: [re.sub(pattern, "", y) for y in x])
    all_feat['features'] = all_feat['features'].apply(lambda x: [stemmer.stem(y) for y in x])
    all_feat['features'] = all_feat['features'].apply(lambda x: ['_'.join(['feature'] + y.split()) for y in x])
    all_feat['features'] = all_feat['features'].apply(lambda x: ' '.join(x))
    vect_df = most_freq_vects(all_feat['features'], max_feature=100, token_pattern=r"[^ ]+")
    
    all_feat = pd.concat([all_feat, vect_df], axis=1)
    train = all_feat[all_feat.interest_level != -1].copy()
    test = all_feat[all_feat.interest_level == -1].copy()
    y_train=train["interest_level"]

    x_train = train.drop(["interest_level","features"],axis=1)
    x_test = test.drop(["interest_level","features"],axis=1)

    return x_train, y_train, x_test, x_test.columns.values, x_test.listing_id


def _preprocess(dtrain, dtest):
    # replace np.inf to np.nan
    dtrain = dtrain.replace([np.inf, -np.inf], np.nan)
    dtest = dtest.replace([np.inf, -np.inf], np.nan)

    # impute np.nan
    dtrain_col_mean = dtrain.mean(axis=0)
    dtrain, dtest = dtrain.fillna(dtrain_col_mean), dtest.fillna(dtrain_col_mean)

    # perform standardization
    dtrain_col_mean, dtrain_col_std = dtrain.mean(axis=0), dtrain.std(axis=0)
    dtrain, dtest = map(lambda x: (x - dtrain_col_mean) / dtrain_col_std, (dtrain, dtest))

    return dtrain, dtest


def _preprocess_log(dtrain, dtest):
    # replace np.inf to np.nan
    dtrain = dtrain.replace([np.inf, -np.inf], np.nan)
    dtest = dtest.replace([np.inf, -np.inf], np.nan)

    # impute np.nan
    dtrain_col_mean = dtrain.mean(axis=0)
    dtrain, dtest = dtrain.fillna(dtrain_col_mean), dtest.fillna(dtrain_col_mean)

    # log transform of min-zero columns
    dtrain_col_min = dtrain.min(axis=0)
    zero_min_index = dtrain_col_min[dtrain_col_min >= 0].index

    dtrain[zero_min_index] = np.log10(dtrain[zero_min_index] + 1.0)
    dtest[zero_min_index] = np.log10(dtest[zero_min_index] + 1.0)

    # perform standardization
    dtrain_col_mean, dtrain_col_std = dtrain.mean(axis=0), dtrain.std(axis=0)
    dtrain, dtest = map(lambda x: (x - dtrain_col_mean) / dtrain_col_std, (dtrain, dtest))

    return dtrain, dtest

Caculate performance using 5-fold cross-validation.

In [3]:
def run_model(dtrain, dtest=None, n_neighbors=5, weights='uniform'):
    clf = KNeighborsClassifier()
    params = {'n_neighbors': n_neighbors,
              'weights': weights,
              'n_jobs': -1,
              }
    clf.set_params(**params)
    if dtest:
        clf.fit(dtrain[0], dtrain[1])
        y_train_pred, y_test_pred = clf.predict_proba(dtrain[0]), clf.predict_proba(dtest[0])
        y_train_loss, y_test_loss = log_loss(dtrain[1], y_train_pred), log_loss(dtest[1], y_test_pred)
        return clf, y_train_loss, y_test_loss
    else:
        clf.fit(dtrain[0], dtrain[1])
        y_train_pred = clf.predict_proba(dtrain[0])
        y_train_loss = log_loss(dtrain[1], y_train_pred)
        return clf, y_train_loss

In [4]:
def train_cv(preprocess='linear', n_neighbors=5, weights='uniform'):
    X_train, y_train_cls, X_test, _, _ = load_data()
    if preprocess == 'log':
        X_train, X_test = _preprocess_log(X_train, X_test)
    else:
        X_train, X_test = _preprocess(X_train, X_test)

    cv_scores, n_folds = [], 5
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=816)
    for i, (train_ind, val_ind) in enumerate(skf.split(X_train, y_train_cls)):
        print("Running Fold", i + 1, "/", n_folds)
        start = time.time()
        
        train_x, val_x = X_train.iloc[train_ind, :], X_train.iloc[val_ind, :]
        train_y, val_y = y_train_cls.iloc[train_ind], y_train_cls.iloc[val_ind]
        clf, train_loss, val_loss = run_model((train_x, train_y), (val_x, val_y), n_neighbors, weights)
        cv_scores.append([train_loss, val_loss])
        
        print("train_loss: {0:.6f}, val_loss: {1:.6f}".format(train_loss, val_loss), end="\t")
        
        end = time.time()
        m, s = divmod(end-start, 60)
        h, m = divmod(m, 60)
        print("time elapsed: %d:%02d:%02d" % (h, m, s))
        
    mean_train_loss = np.mean([cv_scores[i][0] for i in range(len(cv_scores))])
    mean_val_loss = np.mean([cv_scores[i][1] for i in range(len(cv_scores))])
    print("train_loss mean: {0:.6f}, val_loss mean: {1:.6f}".format(mean_train_loss, mean_val_loss))

In [5]:
train_cv(preprocess='log', n_neighbors=200)

Loading features files
Ordinal encoding
Features document-term matrix
Running Fold 1 / 5
train_loss: 0.630175, val_loss: 0.649817	time elapsed: 0:04:27
Running Fold 2 / 5
train_loss: 0.633348, val_loss: 0.640182	time elapsed: 0:03:59
Running Fold 3 / 5
train_loss: 0.631506, val_loss: 0.639124	time elapsed: 0:04:15
Running Fold 4 / 5
train_loss: 0.631418, val_loss: 0.641704	time elapsed: 0:03:50
Running Fold 5 / 5
train_loss: 0.631175, val_loss: 0.647124	time elapsed: 0:04:23
train_loss mean: 0.631524, val_loss mean: 0.643590


Determine optimum num of neighbors using GridSearchCV.

In [12]:
def _GridSearchCV(preprocess='linear', n_neighbors=None):
    X_train, y_train_cls, X_test, _, _ = load_data()
    if preprocess == 'log':
        X_train, X_test = _preprocess_log(X_train, X_test)
    else:
        X_train, X_test = _preprocess(X_train, X_test)

    kneigh = KNeighborsClassifier(n_jobs=-1)
    params_grid = {'n_neighbors': n_neighbors}

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=816)
    clf = GridSearchCV(kneigh, params_grid, scoring='neg_log_loss', n_jobs=-1, verbose=2, 
                       cv=skf.split(X_train, y_train_cls))
    clf.fit(X_train, y_train_cls)
    return clf

In [14]:
clf = _GridSearchCV(preprocess='log', n_neighbors=[1,10,100,1000])

Loading features files
Ordinal encoding
Features document-term matrix
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] n_neighbors=1 ...................................................
[CV] n_neighbors=1 ...................................................
[CV] n_neighbors=1 ...................................................
[CV] n_neighbors=1 ...................................................
[CV] .................................... n_neighbors=1, total= 2.3min
[CV] n_neighbors=1 ...................................................
[CV] .................................... n_neighbors=1, total= 2.3min
[CV] n_neighbors=10 ..................................................
[CV] .................................... n_neighbors=1, total= 2.4min
[CV] n_neighbors=10 ..................................................
[CV] .................................... n_neighbors=1, total= 2.3min
[CV] n_neighbors=10 ..................................................
[CV] .............

[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 62.3min finished


In [15]:
clf.cv_results_.keys()

dict_keys(['split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'split4_train_score', 'mean_train_score', 'std_train_score', 'mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_n_neighbors', 'params'])

In [22]:
# the code is to optimize the negative logloss
# thus take negative gives the true logloss
-clf.cv_results_['mean_test_score']

array([ 11.21362486,   1.48584867,   0.6483376 ,   0.67118175])

In [21]:
-clf.cv_results_['mean_train_score']

array([  2.10942375e-15,   5.00982137e-01,   6.15775669e-01,
         6.69489245e-01])

In [17]:
clf.cv_results_['params']

({'n_neighbors': 1},
 {'n_neighbors': 10},
 {'n_neighbors': 100},
 {'n_neighbors': 1000})

In [26]:
# in seconds
clf.cv_results_['mean_fit_time']

array([ 1.25838928,  2.37082696,  2.61748352,  2.61251588])

In [27]:
# in minutes
clf.cv_results_['mean_score_time'] / 60

array([ 2.2971593 ,  2.74915991,  3.12078474,  3.34374926])

From the GridSearchCV result above, the best score is obtained around $k=100$ nearest neighbors. Thus $k\approx 200$ is good choice.